# Classification   $ $

<hr>
<em> Solving the classification problem of EEG signals associted with a breaking event.<br>
First the signals are divided in sub intervales and we look differents statistics.<br>
The we input thee extract features from the series and use them with normal supervised learning.
</em> 
<hr>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from advisedlib.extraction.csv import CsvExtractor
from advisedlib.dataframe import SynchronousPandasDataFrame, AsynchronousPandasDataFrame, StaticPandasDataFrame
#from advisedlib.feature_selection.ls import LS
#from advisedlib.feature_extraction.tsfresh_feature_extractor import Tsfresh
from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features, select_features

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import roc_auc_score as AUC, accuracy_score as accuracy

In [2]:
path_to_data = os.path.normpath(os.path.join('/Users/obando_c/Documents/Neuro/EmergencyBraking/data/post_processed/Balanced_01'))
files = sorted([elt for elt in os.listdir(path_to_data) if elt[-4:] == '.npy'],
                   key=(lambda x: int(x.split('_')[2])))
np.random.RandomState(seed=0).shuffle(files)
eeg_channels_names = ['Fp1', 'Fp2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 
 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 
 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'P9', 'P7',
 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 
 'O1', 'Oz', 'O2']

In [3]:
spatial_filtering_w_np = np.load("spatial_filtering_w_np.npy")

In [4]:
# DO not run
df_eeg_w = pd.DataFrame([])
labels = []
time_names = ['t'+ str(x) for x in range(319 + 1)]
for num, file in enumerate(files): # create a pandas df with 
    label = {'event': 1.0, 'noevent': 0.0}[file.split('_')[-1].split('.')[0]]
    mat_pp = np.load(os.path.join(path_to_data,file)).T
    #spatial_filtering_w_np = spatial_filtering_w.numpy().reshape(59,1)
    #spatial_filtering_w_np = np.ones(59).reshape(59,1)
    virtual_ch = np.dot(mat_pp ,spatial_filtering_w_np).T
    #data = np.load(os.path.join(path_to_data,file))
    df_interval = pd.DataFrame(data = virtual_ch)
    df_interval.insert(0, "seq_id", num, True) 
    df_interval.insert(1, "event", label, True) 
    df_interval.insert(2, "param", 'eeg_mean', True) 
    df_interval.columns = ["seq_id", "event", "param"] + time_names
    
    df_eeg_w = df_eeg_w.append(df_interval)
    
    labels.append(label)
df_eeg_w.to_csv(r'df_eeg_w.csv', index = False)
# save labels labels.to_csv(r'labels.csv', index = False)

In [5]:
filepath = "df_eeg_w.csv"
csvextractor = CsvExtractor(
    advised_pandas_dataframe_class=SynchronousPandasDataFrame,
    filepath_or_buffer=filepath,
    kwargs_pandas_read_csv={},
    context_columns=["seq_id", "event"],
    context_id_columns=["seq_id"],
    #time_mesh= NPtime.T, #4.0e-06  # ns
    sampling_rate = 200, # 4 Like in tutorial
    #sampling_period = 5.0e-03 #
)
sync_df, context_df = csvextractor.extract()

display(sync_df)
display(context_df)

,seq_id,param,t0,t1,t2,t3,t4,t5,t6,t7,...,t310,t311,t312,t313,t314,t315,t316,t317,t318,t319
0,0,eeg_mean,7.774373,7.522858,6.509889,5.037838,3.797448,2.825851,1.714820,0.085826,...,2.636654,5.418634,5.863342,3.415605,-0.047639,-2.127166,-2.408980,-2.599260,-3.992599,-5.773746
1,1,eeg_mean,-6.226930,-4.997166,-4.349439,-4.961937,-5.645273,-4.563182,-1.092711,3.204452,...,2.478368,1.611020,1.304819,3.470677,8.479639,14.734532,19.270718,19.701118,16.042629,11.094939
2,2,eeg_mean,4.170941,-1.064026,-5.010014,-6.115751,-4.384724,-1.331954,0.992930,1.806779,...,13.420618,11.700576,7.744785,3.599297,2.139112,4.440758,8.777546,12.493044,14.489628,15.403749
3,3,eeg_mean,2.865388,0.390578,-2.189610,-4.960886,-8.377027,-11.321410,-11.257582,-6.259507,...,24.097286,22.069897,17.974409,13.406804,9.974896,8.129632,6.944408,5.458346,3.568830,1.445499
4,4,eeg_mean,4.715312,5.448199,6.772654,6.909220,5.064749,2.190498,-0.474924,-2.574999,...,7.604112,5.420214,5.190669,7.538933,11.592326,15.409449,17.310918,17.248502,16.300778,15.464062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,455,eeg_mean,8.122852,6.811002,3.408504,0.247278,-0.917677,-0.107596,1.267075,1.667013,...,-5.534420,-7.670270,-8.777799,-9.356057,-10.393878,-11.550956,-11.867479,-11.224832,-10.441029,-10.335564
456,456,eeg_mean,2.117522,-0.897782,-1.779986,-0.880579,0.298675,1.153129,1.978397,3.539799,...,-5.074462,-3.129280,-2.943317,-5.157377,-8.301618,-10.321116,-10.607894,-10.364347,-10.728473,-10.874009
457,457,eeg_mean,-5.331190,-1.901850,3.216490,6.616178,6.680736,4.809575,4.126730,5.628881,...,-8.167682,-11.357375,-12.868074,-10.556461,-5.853167,-1.928370,0.136676,1.551716,3.364688,4.872046
458,458,eeg_mean,-3.163641,-2.671629,-2.369507,-2.179532,-1.955868,-1.860586,-2.399992,-3.799630,...,10.974319,9.941296,9.025718,7.551006,5.549130,4.102834,4.425456,6.108094,7.277900,6.585743


,seq_id,event
0,0,1.0
1,1,0.0
2,2,1.0
3,3,1.0
4,4,0.0
...,...,...
455,455,1.0
456,456,0.0
457,457,1.0
458,458,0.0


In [6]:
print(sync_df.shape)
#print(df_eeg.T.shape)
#print(sync_df.time_mesh)
len(sync_df.time_mesh)

(460, 322)


320

In [7]:
params = sync_df.param.drop_duplicates().tolist()
print(params)
assert len(params) == 1

['eeg_mean']


In [8]:
sync_df.check()

,seq_id,param,t0,t1,t2,t3,t4,t5,t6,t7,...,t310,t311,t312,t313,t314,t315,t316,t317,t318,t319
0,0,eeg_mean,7.774373,7.522858,6.509889,5.037838,3.797448,2.825851,1.714820,0.085826,...,2.636654,5.418634,5.863342,3.415605,-0.047639,-2.127166,-2.408980,-2.599260,-3.992599,-5.773746
1,1,eeg_mean,-6.226930,-4.997166,-4.349439,-4.961937,-5.645273,-4.563182,-1.092711,3.204452,...,2.478368,1.611020,1.304819,3.470677,8.479639,14.734532,19.270718,19.701118,16.042629,11.094939
2,2,eeg_mean,4.170941,-1.064026,-5.010014,-6.115751,-4.384724,-1.331954,0.992930,1.806779,...,13.420618,11.700576,7.744785,3.599297,2.139112,4.440758,8.777546,12.493044,14.489628,15.403749
3,3,eeg_mean,2.865388,0.390578,-2.189610,-4.960886,-8.377027,-11.321410,-11.257582,-6.259507,...,24.097286,22.069897,17.974409,13.406804,9.974896,8.129632,6.944408,5.458346,3.568830,1.445499
4,4,eeg_mean,4.715312,5.448199,6.772654,6.909220,5.064749,2.190498,-0.474924,-2.574999,...,7.604112,5.420214,5.190669,7.538933,11.592326,15.409449,17.310918,17.248502,16.300778,15.464062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,455,eeg_mean,8.122852,6.811002,3.408504,0.247278,-0.917677,-0.107596,1.267075,1.667013,...,-5.534420,-7.670270,-8.777799,-9.356057,-10.393878,-11.550956,-11.867479,-11.224832,-10.441029,-10.335564
456,456,eeg_mean,2.117522,-0.897782,-1.779986,-0.880579,0.298675,1.153129,1.978397,3.539799,...,-5.074462,-3.129280,-2.943317,-5.157377,-8.301618,-10.321116,-10.607894,-10.364347,-10.728473,-10.874009
457,457,eeg_mean,-5.331190,-1.901850,3.216490,6.616178,6.680736,4.809575,4.126730,5.628881,...,-8.167682,-11.357375,-12.868074,-10.556461,-5.853167,-1.928370,0.136676,1.551716,3.364688,4.872046
458,458,eeg_mean,-3.163641,-2.671629,-2.369507,-2.179532,-1.955868,-1.860586,-2.399992,-3.799630,...,10.974319,9.941296,9.025718,7.551006,5.549130,4.102834,4.425456,6.108094,7.277900,6.585743


In [9]:
context_df.event.value_counts()

0.0    230
1.0    230
Name: event, dtype: int64

## Feature extraction: tsfresh + selector
[Tsfresh](https://tsfresh.readthedocs.io) is a python library that is designed to extract automatically a lot of features from a timeseries. 

A wrapper for tsfresh extractor is embedded in advisedlib. It can be coupled with a feature selection step in order to reduce the number of generated features and to be able to handle them afterwards with an anomaly scorer.

### Conversion to asynchronous format
The tsfresh wrapper needs an asynchronous input, so we first convert the dataset to asynchronous format.
See the [cookbook](https://docs.cognitive.bocacc.io/advised-lib/notebooks/cookbook.html#Conversion-between-advised-formats) for an example.

In [10]:
async_df = sync_df.to_asynchronous()
display(async_df)

,seq_id,param,time,value
0,0,eeg_mean,2000-01-01 00:00:00.000,7.774373
1,0,eeg_mean,2000-01-01 00:00:00.005,7.522858
2,0,eeg_mean,2000-01-01 00:00:00.010,6.509889
3,0,eeg_mean,2000-01-01 00:00:00.015,5.037838
4,0,eeg_mean,2000-01-01 00:00:00.020,3.797448
...,...,...,...,...
147195,459,eeg_mean,2000-01-01 00:00:01.575,14.212064
147196,459,eeg_mean,2000-01-01 00:00:01.580,15.668113
147197,459,eeg_mean,2000-01-01 00:00:01.585,18.035304
147198,459,eeg_mean,2000-01-01 00:00:01.590,20.842178


In [14]:
# There seems to be a problem with the # of jobs
f = extract_features( async_df, column_id = "seq_id", column_sort = "time", column_kind='param', column_value="value", n_jobs=0 )




Feature Extraction:   0%|          | 0/460 [00:00<?, ?it/s]


Feature Extraction:   0%|          | 1/460 [00:00<01:43,  4.44it/s]


Feature Extraction:   0%|          | 2/460 [00:00<01:47,  4.27it/s]


Feature Extraction:   1%|          | 3/460 [00:00<01:47,  4.26it/s]


Feature Extraction:   1%|          | 4/460 [00:00<01:49,  4.18it/s]


Feature Extraction:   1%|          | 5/460 [00:01<01:48,  4.19it/s]


Feature Extraction:   1%|▏         | 6/460 [00:01<01:47,  4.21it/s]


Feature Extraction:   2%|▏         | 7/460 [00:01<01:47,  4.22it/s]


Feature Extraction:   2%|▏         | 8/460 [00:01<01:46,  4.24it/s]


Feature Extraction:   2%|▏         | 9/460 [00:02<01:46,  4.22it/s]


Feature Extraction:   2%|▏         | 10/460 [00:02<01:46,  4.21it/s]


Feature Extraction:   2%|▏         | 11/460 [00:02<01:46,  4.22it/s]


Feature Extraction:   3%|▎         | 12/460 [00:02<01:45,  4.24it/s]


Feature Extraction:   3%|▎         | 13/460 [00:03<01:45,  4.25it/s]


Feature Extraction:  

Feature Extraction:  44%|████▍     | 203/460 [00:50<01:04,  3.97it/s]


Feature Extraction:  44%|████▍     | 204/460 [00:51<01:04,  3.96it/s]


Feature Extraction:  45%|████▍     | 205/460 [00:51<01:04,  3.98it/s]


Feature Extraction:  45%|████▍     | 206/460 [00:51<01:03,  3.98it/s]


Feature Extraction:  45%|████▌     | 207/460 [00:51<01:05,  3.89it/s]


Feature Extraction:  45%|████▌     | 208/460 [00:52<01:04,  3.91it/s]


Feature Extraction:  45%|████▌     | 209/460 [00:52<01:03,  3.93it/s]


Feature Extraction:  46%|████▌     | 210/460 [00:52<01:03,  3.95it/s]


Feature Extraction:  46%|████▌     | 211/460 [00:52<01:02,  3.98it/s]


Feature Extraction:  46%|████▌     | 212/460 [00:53<01:02,  3.98it/s]


Feature Extraction:  46%|████▋     | 213/460 [00:53<01:01,  4.00it/s]


Feature Extraction:  47%|████▋     | 214/460 [00:53<01:01,  3.99it/s]


Feature Extraction:  47%|████▋     | 215/460 [00:53<01:01,  3.99it/s]


Feature Extraction:  47%|████▋     | 216/460 [00:54<01:01,  3.98

Feature Extraction:  88%|████████▊ | 405/460 [01:42<00:14,  3.85it/s]


Feature Extraction:  88%|████████▊ | 406/460 [01:42<00:14,  3.83it/s]


Feature Extraction:  88%|████████▊ | 407/460 [01:43<00:13,  3.83it/s]


Feature Extraction:  89%|████████▊ | 408/460 [01:43<00:13,  3.81it/s]


Feature Extraction:  89%|████████▉ | 409/460 [01:43<00:13,  3.83it/s]


Feature Extraction:  89%|████████▉ | 410/460 [01:43<00:13,  3.84it/s]


Feature Extraction:  89%|████████▉ | 411/460 [01:44<00:12,  3.88it/s]


Feature Extraction:  90%|████████▉ | 412/460 [01:44<00:12,  3.92it/s]


Feature Extraction:  90%|████████▉ | 413/460 [01:44<00:11,  3.92it/s]


Feature Extraction:  90%|█████████ | 414/460 [01:44<00:11,  3.93it/s]


Feature Extraction:  90%|█████████ | 415/460 [01:45<00:11,  3.96it/s]


Feature Extraction:  90%|█████████ | 416/460 [01:45<00:11,  3.94it/s]


Feature Extraction:  91%|█████████ | 417/460 [01:45<00:10,  3.96it/s]


Feature Extraction:  91%|█████████ | 418/460 [01:45<00:10,  3.91

In [15]:
impute( f )
assert f.isnull().sum().sum() == 0

In [16]:
features_file = "df_eeg_w_features.csv"

In [17]:
f['y'] = labels
f.to_csv( features_file, index = None )

In [18]:
features = pd.read_csv( features_file )

In [19]:
validation_split_i = int(features.shape[0]/2)

In [20]:
train_x = features.iloc[:validation_split_i].drop( 'y', axis = 1 )
test_x = features.iloc[validation_split_i:].drop( 'y', axis = 1 )

train_y = features.iloc[:validation_split_i].y
test_y = features.iloc[validation_split_i:].y

In [21]:
train_features_selected = select_features( train_x, train_y, fdr_level = 0.05 )

In [22]:
print(train_features_selected.columns)

Index(['eeg_mean__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"stderr"',
       'eeg_mean__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"stderr"',
       'eeg_mean__spkt_welch_density__coeff_2',
       'eeg_mean__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"stderr"',
       'eeg_mean__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"',
       'eeg_mean__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"stderr"',
       'eeg_mean__autocorrelation__lag_3', 'eeg_mean__autocorrelation__lag_2',
       'eeg_mean__autocorrelation__lag_1',
       'eeg_mean__partial_autocorrelation__lag_1',
       ...
       'eeg_mean__partial_autocorrelation__lag_4', 'eeg_mean__skewness',
       'eeg_mean__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"',
       'eeg_mean__fft_coefficient__coeff_3__attr_"imag"',
       'eeg_mean__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',
       'eeg_mean__fft_aggregated__aggtype_"skew"',
       'eeg_mean__fft_coeffici

In [23]:
train = train_features_selected.copy()
train['y'] = train_y

test = test_x[ train_features_selected.columns ].copy()
test['y'] = test_y

In [24]:
train_file = "df_eeg_w_train.csv"
test_file = "df_eeg_w_test.csv"

In [25]:
train.to_csv( train_file, index = None )
test.to_csv( test_file, index = None )

In [26]:
train = pd.read_csv( train_file )
test = pd.read_csv( test_file )

x_train = train.drop( 'y', axis = 1 ).values
y_train = train.y.values

x_test = test.drop( 'y', axis = 1 ).values
y_test = test.y.values

In [27]:
classifiers = [
#LR( C = 10 ),
#LR( C = 1 ),
#LR( C = 0.1 ),
    
#make_pipeline( StandardScaler(), LR()),	
#make_pipeline( StandardScaler(), LR( C = 10 )),
make_pipeline( StandardScaler(), LR( C = 30 )),

#make_pipeline( MinMaxScaler(), LR()),
#make_pipeline( MinMaxScaler(), LR( C = 10 )),	
make_pipeline( MinMaxScaler(), LR( C = 30 )),

#LDA(),
RF( n_estimators = 100, min_samples_leaf = 5 )
]

In [28]:
for clf in classifiers:
    clf.fit( x_train, y_train )
    p = clf.predict_proba( x_test )[:,1]
    p_bin = clf.predict( x_test )
    
    auc = AUC( y_test, p )
    acc = accuracy( y_test, p_bin )
    print( "AUC: {:.2%}, accuracy: {:.2%} \n\n{}\n\n".format( auc, acc, clf ))


AUC: 97.78%, accuracy: 96.96% 

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logisticregression',
                 LogisticRegression(C=30, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)


AUC: 98.16%, accuracy: 96.52% 

Pipeline(memory=None,
         steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('logisticregression',
                 LogisticRegression(C=30, class_weight=None, dual=False,
                                    fi

/Users/obando_c/anaconda3/envs/advisedlib-env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AUC: 99.24%, accuracy: 96.09% 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)




In [ ]:
# The NN does a better job in terms of accuracy 

In [ ]:
# What if I collapse all the electrodes into one cirtual one and then extract features in thet signal